In [12]:
import os, json, subprocess, re, warnings
from pathlib import Path

warnings.filterwarnings("ignore")

# Adjust if notebook is elsewhere, but this matches your current layout
FFMPEG_DIR = Path("./") / "ffmpeg-2025-12-10-git-4f947880bd-essentials_build" / "bin"
FFMPEG_BIN = str((FFMPEG_DIR / "ffmpeg.exe").resolve())
FFPROBE_BIN = str((FFMPEG_DIR / "ffprobe.exe").resolve())

print("FFMPEG_BIN =", FFMPEG_BIN)
print("FFPROBE_BIN =", FFPROBE_BIN)

# Folder with your 50 MP3s
MP3_FOLDER = Path(r"D:/Downloads/Harry Potter - Oh God Not Again! FanFiction Patronus Pages AudioBook").resolve()
print("MP3_FOLDER =", MP3_FOLDER)

# Final output name (base)
DEFAULT_OUTPUT_NAME = "Oh_God_Not_Again_Harry_Potter_FanFic.m4b"

METADATA = {
    "title": "Oh God Not Again! Harry Potter FanFic",
    "artist": "Sarah1281",
    "author": "Sarah1281",
    "album": "Oh God Not Again! Harry Potter FanFic Audiobook",
    "album_artist": "Patronus Pages",
    "genre": "Audiobook",
    "date": "2008",
    "comment": (
        "Patronus Pages YT audiobook: "
        "https://www.youtube.com/playlist?list=PLba8WLPdVZSrCgji3KUyDuHpYI3mVruZD "
        "based on Harry Potter FanFic: "
        "https://www.fanfiction.net/s/4536005/2/Oh-God-Not-Again"
    ),
}

def get_duration(file_path: Path) -> float:
    """Get audio duration in seconds using ffprobe."""
    cmd = [
        FFPROBE_BIN, "-v", "quiet", "-print_format", "json",
        "-show_format", str(file_path)
    ]
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    data = json.loads(result.stdout)
    return float(data["format"]["duration"])

def parse_chapter_number(filename: str) -> int:
    """Extract chapter number from 'Chapter X' pattern in filename."""
    match = re.search(r"Chapter\s+(\d+)", filename, re.IGNORECASE)
    return int(match.group(1)) if match else float("inf")


FFMPEG_BIN = D:\projects\mp3tom4b_audiobook\ffmpeg-2025-12-10-git-4f947880bd-essentials_build\bin\ffmpeg.exe
FFPROBE_BIN = D:\projects\mp3tom4b_audiobook\ffmpeg-2025-12-10-git-4f947880bd-essentials_build\bin\ffprobe.exe
MP3_FOLDER = D:\Downloads\Harry Potter - Oh God Not Again! FanFiction Patronus Pages AudioBook


In [ ]:
#Recode mp3 and create intermediate files

folder = MP3_FOLDER
print("Using MP3 folder:", folder)

mp3_files = list(folder.glob("*.mp3"))
if not mp3_files:
    raise SystemExit("No MP3 files found in folder")

# Sort by chapter number in filename
mp3_files.sort(key=lambda f: parse_chapter_number(f.name))

print(f"Found {len(mp3_files)} MP3 files (sorted by chapter):")
for i, mp3 in enumerate(mp3_files, 1):
    chap_num = parse_chapter_number(mp3.name)
    print(f"  {i:02d}. Chapter {chap_num if chap_num != float('inf') else '?'} -> {mp3.name}")

# 1a. Create concat list
concat_file = "concat.txt"
with open(concat_file, "w", encoding="utf-8") as f:
    for mp3 in mp3_files:
        f.write(f"file '{mp3.as_posix()}'\n")

print("Wrote concat list to:", concat_file)

# 1b. Create FFmetadata with chapters
ffmeta_file = "chapters.ffmetadata"
current_time = 0.0

with open(ffmeta_file, "w", encoding="utf-8") as f:
    f.write(";FFMETADATA1\n")
    for i, mp3 in enumerate(mp3_files, 1):
        duration = get_duration(mp3)
        start_ms = int(current_time * 1000)
        end_ms = int((current_time + duration) * 1000)

        chap_num = parse_chapter_number(mp3.name)
        if chap_num == float("inf"):
            chap_num = i
        chapter_title = f"Chapter {chap_num}"

        f.write("[CHAPTER]\n")
        f.write("TIMEBASE=1/1000\n")
        f.write(f"START={start_ms}\n")
        f.write(f"END={end_ms}\n")
        f.write(f"title={chapter_title}\n\n")

        current_time += duration
        print(f"  -> {chapter_title}: {duration/60:.1f} min")

print("Wrote chapter metadata to:", ffmeta_file)

# 1c. Concatenate MP3s to temporary M4A (AAC 80 kb/s mono)
temp_m4a = "temp_combined.m4a"
print("\n[Step 1] Concatenating MP3 files into AAC M4A (80 kb/s mono)...")
cmd = [
    FFMPEG_BIN, "-y",
    "-f", "concat", "-safe", "0",
    "-i", str(concat_file),
    "-ac", "1",          # mono
    "-c:a", "aac",
    "-b:a", "80k",       # target bitrate
    str(temp_m4a),
]
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True)

print("Created:", temp_m4a)
print(f"Total duration ~ {current_time/3600:.2f} hours")


Using MP3 folder: D:\Downloads\Harry Potter - Oh God Not Again! FanFiction Patronus Pages AudioBook
Found 50 MP3 files (sorted by chapter):
  01. Chapter 1 -> Harry Potter - Oh God Not Again!  Chapter 1 _ FanFiction AudioBook.mp3
  02. Chapter 2 -> Harry Potter - Oh God Not Again!  Chapter 2 _ FanFiction AudioBook.mp3
  03. Chapter 3 -> Harry Potter - Oh God Not Again!  Chapter 3 _ FanFiction AudioBook.mp3
  04. Chapter 4 -> Harry Potter - Oh God Not Again!  Chapter 4 _ FanFiction AudioBook.mp3
  05. Chapter 5 -> Harry Potter - Oh God Not Again!  Chapter 5 _ FanFiction AudioBook.mp3
  06. Chapter 6 -> Harry Potter - Oh God Not Again!  Chapter 6 _ FanFiction AudioBook.mp3
  07. Chapter 7 -> Harry Potter - Oh God Not Again!  Chapter 7 _ FanFiction AudioBook.mp3
  08. Chapter 8 -> Harry Potter - Oh God Not Again!  Chapter 8 _ FanFiction AudioBook.mp3
  09. Chapter 9 -> Harry Potter - Oh God Not Again!  Chapter 9 _ FanFiction AudioBook.mp3
  10. Chapter 10 -> Harry Potter - Oh God Not Agai

In [ ]:
#Combine into one m4b with chapters

from pathlib import Path
import subprocess

AUDIO_IN   = "temp_combined.m4a"
FFMETA_IN  = "chapters.ffmetadata"
OUTPUT_M4B = "_Oh_God_Not_Again_Harry_Potter_FanFic_no_cover.m4b"

print("Audio in:  ", AUDIO_IN)
print("Meta in:   ", FFMETA_IN)
print("Output m4b:", OUTPUT_M4B)

cmd = [
    FFMPEG_BIN, "-y",
    "-i", str(AUDIO_IN),      # 0: audio
    "-i", str(FFMETA_IN),     # 1: chapters
    "-map_metadata", "1",     # take chapters/metadata from ffmetadata
    "-c", "copy",
    "-metadata", f"title={METADATA['title']}",
    "-metadata", f"artist={METADATA['artist']}",
    "-metadata", f"author={METADATA['author']}",
    "-metadata", f"album={METADATA['album']}",
    "-metadata", f"album_artist={METADATA['album_artist']}",
    "-metadata", f"genre={METADATA['genre']}",
    "-metadata", f"date={METADATA['date']}",
    "-metadata", f"comment={METADATA['comment']}",
    str(OUTPUT_M4B),
]

print("Running:", " ".join(cmd))
result = subprocess.run(cmd, text=True, capture_output=True)
print("\n=== FFmpeg stdout ===\n", result.stdout)
print("\n=== FFmpeg stderr ===\n", result.stderr)
print("\nReturn code:", result.returncode)


Audio in:   temp_combined.m4a
Meta in:    chapters.ffmetadata
Output m4b: _Oh_God_Not_Again_Harry_Potter_FanFic_no_cover.m4b
Running: D:\projects\mp3tom4b_audiobook\ffmpeg-2025-12-10-git-4f947880bd-essentials_build\bin\ffmpeg.exe -y -i temp_combined.m4a -i chapters.ffmetadata -map_metadata 1 -c copy -metadata title=Oh God Not Again! Harry Potter FanFic -metadata artist=Sarah1281 -metadata author=Sarah1281 -metadata album=Oh God Not Again! Harry Potter FanFic Audiobook -metadata album_artist=Patronus Pages -metadata genre=Audiobook -metadata date=2008 -metadata comment=Patronus Pages YT audiobook: https://www.youtube.com/playlist?list=PLba8WLPdVZSrCgji3KUyDuHpYI3mVruZD based on Harry Potter FanFic: https://www.fanfiction.net/s/4536005/2/Oh-God-Not-Again _Oh_God_Not_Again_Harry_Potter_FanFic_no_cover.m4b

=== FFmpeg stdout ===
 

=== FFmpeg stderr ===
 ffmpeg version 2025-12-10-git-4f947880bd-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.

In [ ]:
#Add embeded cover

import subprocess
from pathlib import Path

folder = MP3_FOLDER

INPUT_M4B  = "_Oh_God_Not_Again_Harry_Potter_FanFic_no_cover.m4b"
COVER_IMG  = folder / "cover_thumbnail.jpg"    # or .png
OUTPUT_M4B = folder / "Oh_God_Not_Again_Harry_Potter_FanFic.m4b"

print("Input m4b: ", INPUT_M4B)
print("Cover:     ", COVER_IMG)
print("Output m4b:", OUTPUT_M4B)

if not COVER_IMG.exists():
    raise SystemExit(f"Cover image not found: {COVER_IMG}")

cmd = [
    FFMPEG_BIN, "-y",
    "-i", str(INPUT_M4B),      # 0: audio+chapters
    "-i", str(COVER_IMG),      # 1: cover
    "-map", "0:a",             # audio
    "-map", "0:t?",            # existing chapters/metadata
    "-map", "1:v",             # cover image
    "-c", "copy",
    "-c:v", "mjpeg" if COVER_IMG.suffix.lower() in [".jpg", ".jpeg"] else "png",
    "-disposition:v:0", "attached_pic",
    str(OUTPUT_M4B),
]

print("Running:", " ".join(cmd))
result = subprocess.run(cmd, text=True, capture_output=True)
print("\n=== FFmpeg stdout ===\n", result.stdout)
print("\n=== FFmpeg stderr ===\n", result.stderr)
print("\nReturn code:", result.returncode)


Input m4b:  _Oh_God_Not_Again_Harry_Potter_FanFic_no_cover.m4b
Cover:      D:\Downloads\Harry Potter - Oh God Not Again! FanFiction Patronus Pages AudioBook\cover_thumbnail.jpg
Output m4b: D:\Downloads\Harry Potter - Oh God Not Again! FanFiction Patronus Pages AudioBook\Oh_God_Not_Again_Harry_Potter_FanFic.m4b
Running: D:\projects\mp3tom4b_audiobook\ffmpeg-2025-12-10-git-4f947880bd-essentials_build\bin\ffmpeg.exe -y -i _Oh_God_Not_Again_Harry_Potter_FanFic_no_cover.m4b -i D:\Downloads\Harry Potter - Oh God Not Again! FanFiction Patronus Pages AudioBook\cover_thumbnail.jpg -map 0:a -map 0:t? -map 1:v -c copy -c:v mjpeg -disposition:v:0 attached_pic D:\Downloads\Harry Potter - Oh God Not Again! FanFiction Patronus Pages AudioBook\Oh_God_Not_Again_Harry_Potter_FanFic.m4b

=== FFmpeg stdout ===
 

=== FFmpeg stderr ===
 ffmpeg version 2025-12-10-git-4f947880bd-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)


In [ ]:
#Inspect any .m4b fole for stats

FFPROBE_BIN = r"D:\projects\mp3tom4b_audiobook\ffmpeg-2025-12-10-git-4f947880bd-essentials_build\bin\ffprobe.exe"
FILE = Path(r"./The Black Prism.m4b")

result = subprocess.run([
    FFPROBE_BIN,
    "-v", "error",
    "-show_entries", "format=duration,bit_rate",
    "-show_entries", "stream=index,codec_name,codec_type,channels,sample_rate",
    "-of", "json",
    str(FILE)
], text=True, capture_output=True)

print(result.stdout)